# STA 220 Data & Web Technologies for Data Analysis


### Lecture 4, 1/16/25, APIs


### Today's topics

- Undocumented APIs

### Ressources
 - [Yolo County Health Inspections](https://yoloeco.envisionconnect.com/)

### Recap: HTTP

A response to an HTTP request always includes a status code that summarizes whether the request was successful. Wikipedia has a full [list of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). Generally,

* 200-299: Your request succeeded.
* 300-399: You need to take further action to complete the request.
* 400-499: Your request wasn't valid (you made a mistake). You've probably seen 404 before!
* 500-599: Your request failed (the server made a mistake).

In [1]:
import requests

### Undocumented Web APIs

Many websites use undocumented web APIs to get data. For example:

 - [University of California Compensation](https://ucannualwage.ucop.edu/wage/)
 - [Yolo County Health Inspections](https://yoloeco.envisionconnect.com/)

You can identify these websites by looking at requests in your browser's developer tools. For Firefox and Chrome these can be accessed (Windows: <kbd>Ctrl</kbd> + <kbd>i</kbd>; MacOS: <kbd>&#8984;</kbd> + <kbd>&#8997;</kbd> + <kbd>i</kbd>).

Requests to web APIs almost always return JSON or XML data. By examining the browser requests, you can work out the endpoints and parameters, allowing you to use the API.

**CAUTION:** Web APIs that are undocumented are often undocumented for a reason. Using an undocumented API may make someone angry or get you into legal trouble! Government and quasi-government websites (like the examples above) are probably okay, as long as you cache and rate-limit your requests. For everything else, find for an alternative or get permission first.

Let's reverse engineer the Yolo County Health Inspections web API so that we can get data about local restaurants.

In [2]:
import numpy as np
import pandas as pd
import requests

In [3]:
url = 'https://yoloeco.envisionconnect.com/api/pressAgentClient/searchFacilities'

In [4]:
result = requests.post(url, 
                       data = {
    'FacilityName': "Ali Baba"
})

In [5]:
result.text

'{"Message":"No HTTP resource was found that matches the request URI \'https://yoloeco.envisionconnect.com/api/pressAgentClient/searchFacilities\'.","MessageDetail":"No action was found on the controller \'PressAgentApi\' that matches the request."}'

Check the [docs](https://requests.readthedocs.io/en/latest/api/?highlight=post#requests.post) for `requests`!

In [6]:
result.url

'https://yoloeco.envisionconnect.com/api/pressAgentClient/searchFacilities'

In [7]:
result.json()

{'Message': "No HTTP resource was found that matches the request URI 'https://yoloeco.envisionconnect.com/api/pressAgentClient/searchFacilities'.",
 'MessageDetail': "No action was found on the controller 'PressAgentApi' that matches the request."}

Lets investigate this further. The second request uses the `FacilityID` as parameter. 

In [8]:
url = 'https://yoloeco.envisionconnect.com/api/pressAgentClient/programs'
result = requests.get(url, params = {
    'FacilityId': 'FA0001973', 
    'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4'
})
result.raise_for_status()
result.json()

[{'ProgramId': 'PR0000674',
  'ProgramIdentifier': None,
  'ProgramCategory': 'FOOD INSPECTION',
  'LastScore': 100.0,
  'attachmentId': 'bbb84fb5-2bf6-475d-8cb0-b0bc01067403'}]

In [9]:
result.url

'https://yoloeco.envisionconnect.com/api/pressAgentClient/programs?FacilityId=FA0001973&PressAgentOid=c08cb189-894c-4c8c-b595-a5ef010226b4'

We are interested in the inspections text, for which we have to provide the `ProgramID` parameter. 

In [10]:
url = 'https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections'

In [11]:
result = requests.get(url, params = {
    'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
    'ProgramId': 'PR0000674'
})
result.raise_for_status()

In [12]:
results = result.json()
results

[{'activity_date': '2023-11-16T00:00:00',
  'service': 'FOLLOWUP INSPECTION',
  'Oid': '05655fce-31a1-4a75-9f96-b0bc00b0c617',
  'FacilityOid': 'a4602905-d0f7-4fbe-8388-7de1163c420e',
  'score': 100.0,
  'attachmentId': 'bbb84fb5-2bf6-475d-8cb0-b0bc01067403',
  'color': 'Green',
  'violations': []},
 {'activity_date': '2023-10-31T00:00:00',
  'service': 'ROUTINE-INSPECTION',
  'Oid': 'b5ca61ea-34af-421c-add5-b0ac00b6f85d',
  'FacilityOid': 'a4602905-d0f7-4fbe-8388-7de1163c420e',
  'score': 100.0,
  'attachmentId': 'd836ecf4-d162-48af-a432-b0ac00f7a210',
  'color': 'Green',
  'violations': [{'violation_description': 'K005 - Hands clean and properly washed gloves used properly',
    'v_memo': 'California Retail Food Code §113953.3. Observed employee dry hands on towel used to wipe the counter and then proceed back into food prep. Inspector asked employee to wash hands prior to going back to food prep. Employee observed cracking eggs and then switch task to food prep without hand washing.

In [13]:
results_df = pd.DataFrame(results)
results_df

,activity_date,service,Oid,FacilityOid,score,attachmentId,color,violations
0,2023-11-16T00:00:00,FOLLOWUP INSPECTION,05655fce-31a1-4a75-9f96-b0bc00b0c617,a4602905-d0f7-4fbe-8388-7de1163c420e,100.0,bbb84fb5-2bf6-475d-8cb0-b0bc01067403,Green,[]
1,2023-10-31T00:00:00,ROUTINE-INSPECTION,b5ca61ea-34af-421c-add5-b0ac00b6f85d,a4602905-d0f7-4fbe-8388-7de1163c420e,100.0,d836ecf4-d162-48af-a432-b0ac00f7a210,Green,[{'violation_description': 'K005 - Hands clean...
2,2023-10-31T00:00:00,COMPLAINT INVESTIGATION,3d61a32b-d75d-4442-ab2d-b0ac00b9ec71,a4602905-d0f7-4fbe-8388-7de1163c420e,100.0,01b6bcba-472a-4720-8a4e-b0ac00f8214e,Green,[]
3,2023-02-28T00:00:00,ROUTINE-INSPECTION,722bd292-ad66-46ad-a16b-afb7011a322e,a4602905-d0f7-4fbe-8388-7de1163c420e,100.0,5323c79d-03c4-418f-bb06-afb800b42912,Green,"[{'violation_description': 'K045 - Floors, wal..."
4,2022-05-09T00:00:00,ROUTINE-INSPECTION,2ff09bcd-9a36-4b8d-ada1-ae90011ebb4d,a4602905-d0f7-4fbe-8388-7de1163c420e,100.0,3fccdc38-5271-4669-bccc-ae9100aa99ee,Green,[{'violation_description': 'K006 - Adequate ha...


In [14]:
results_df['violations'][1]

[{'violation_description': 'K005 - Hands clean and properly washed gloves used properly',
  'v_memo': 'California Retail Food Code §113953.3. Observed employee dry hands on towel used to wipe the counter and then proceed back into food prep. Inspector asked employee to wash hands prior to going back to food prep. Employee observed cracking eggs and then switch task to food prep without hand washing. Employee was asked to wash hands prior to changing task to ready to eat foods. Employees shall wash their hands before handling any food or food related items and after any task that which may cause potential cross contamination. Corrected on site. \r\n',
  'violation_text': 'Employees are required to wash their hands: before beginning work before handling food / equipment / utensils as often as necessary, during food preparation, to remove soil and contamination when switching from working with raw to ready to eat foods, after touching body parts after using toilet room or any time when co

In [15]:
results_df['violations'][1][0]['v_memo']

'California Retail Food Code §113953.3. Observed employee dry hands on towel used to wipe the counter and then proceed back into food prep. Inspector asked employee to wash hands prior to going back to food prep. Employee observed cracking eggs and then switch task to food prep without hand washing. Employee was asked to wash hands prior to changing task to ready to eat foods. Employees shall wash their hands before handling any food or food related items and after any task that which may cause potential cross contamination. Corrected on site. \r\n'

In [16]:
len(results_df['violations'][1])

5

In [17]:
violations = [
    results_df['violations'][1][i]['violation_description'] for i in range(len(results_df['violations'][1]))
]
violations

['K005 - Hands clean and properly washed gloves used properly',
 'K006 - Adequate handwashing facilities supplied & accessible',
 'K039 - Thermometers provided and accurate',
 'K040 - Wiping cloths: properly used and stored',
 'K045 - Floors, walls and ceilings: built, maintained, and clean']

In [18]:
{'Ali Baba': violations}

{'Ali Baba': ['K005 - Hands clean and properly washed gloves used properly',
  'K006 - Adequate handwashing facilities supplied & accessible',
  'K039 - Thermometers provided and accurate',
  'K040 - Wiping cloths: properly used and stored',
  'K045 - Floors, walls and ceilings: built, maintained, and clean']}

How can we generalize this procedure? 

In [19]:
url = 'https://yoloeco.envisionconnect.com/api/pressAgentClient/searchFacilities'

In [20]:
result=requests.post(url, params  = {
    "PressAgentOid": "c08cb189-894c-4c8c-b595-a5ef010226b4"
}, 
                     data = {
    "FacilityName": "Ali Baba", 
})
result.raise_for_status()

In [21]:
result.json()

[{'FacilityId': 'FA0001973',
  'FacilityName': 'ALI BABA RESTAURANT',
  'Address': '220 3RD ST ',
  'CityStateZip': 'DAVIS CA 95616 ',
  'LastScore': 100.0,
  'attachmentId': 'bbb84fb5-2bf6-475d-8cb0-b0bc01067403'}]

In [22]:
result=requests.post(url, params  = {
    "PressAgentOid": "c08cb189-894c-4c8c-b595-a5ef010226b4"}, 
              data = {
    "FacilityName": "a", 
})
result.json()

[{'FacilityId': 'FA0001345',
  'FacilityName': 'A&B LIQUOR',
  'Address': '2328 W CAPITOL AVE ',
  'CityStateZip': 'WEST SACRAMENTO CA 95691 ',
  'LastScore': 100.0,
  'attachmentId': 'c3afdd86-26f1-4cd1-859a-b0f300b80b46'},
 {'FacilityId': 'FA0021935',
  'FacilityName': "AARON'S CHICKEN SHACK",
  'Address': 'YOLO COUNTY FAIR ',
  'CityStateZip': 'WOODLAND CA 95776 ',
  'LastScore': 100.0,
  'attachmentId': 'ac45b3d9-89d1-43ab-909b-b1ce0089ca09'},
 {'FacilityId': 'FA0011391',
  'FacilityName': 'ACAI FRESH',
  'Address': '431 G ST ',
  'CityStateZip': 'DAVIS CA 95616 ',
  'LastScore': 100.0,
  'attachmentId': '170e6176-2b48-4be2-a5fe-b221008fb38f'},
 {'FacilityId': 'FA0022329',
  'FacilityName': 'ACE SUSHI @SAVE MART 604',
  'Address': '1900 ANDERSON RD ',
  'CityStateZip': 'DAVIS CA 95616 ',
  'LastScore': 100.0,
  'attachmentId': '6c94833d-3c62-465c-9b24-b21400b6061d'},
 {'FacilityId': 'FA0019474',
  'FacilityName': 'ACOUSTIC EVENTS',
  'Address': '918 S S ST ',
  'CityStateZip': 'SAC

In [23]:
pd.DataFrame(result.json())

,FacilityId,FacilityName,Address,CityStateZip,LastScore,attachmentId
0,FA0001345,A&B LIQUOR,2328 W CAPITOL AVE,WEST SACRAMENTO CA 95691,100.0,c3afdd86-26f1-4cd1-859a-b0f300b80b46
1,FA0021935,AARON'S CHICKEN SHACK,YOLO COUNTY FAIR,WOODLAND CA 95776,100.0,ac45b3d9-89d1-43ab-909b-b1ce0089ca09
2,FA0011391,ACAI FRESH,431 G ST,DAVIS CA 95616,100.0,170e6176-2b48-4be2-a5fe-b221008fb38f
3,FA0022329,ACE SUSHI @SAVE MART 604,1900 ANDERSON RD,DAVIS CA 95616,100.0,6c94833d-3c62-465c-9b24-b21400b6061d
4,FA0019474,ACOUSTIC EVENTS,918 S S ST,SACRAMENTO CA 95811,100.0,6ad8b394-103b-4f88-8e1c-ae2b01015500
5,FA0014014,AFC SUSHI / HOT WOK @ BEL AIR #526,1885 E GIBSON RD,WOODLAND CA 95776,100.0,e9d3e0fe-1dfb-4508-8dd1-b1f700a3773a
6,FA0014013,AFC SUSHI / HOT WOK @ RALEY'S #206,367 W MAIN ST,WOODLAND CA 95695,100.0,e52ebc5a-d870-4441-9fff-b18500b38b08
7,FA0014015,AFC SUSHI / HOT WOK @ RALEY'S #448,1601 W CAPITOL AVE,WEST SACRAMENTO CA 95691,100.0,5b9677bc-a18f-41d1-8975-b20b0102d1a1
8,FA0010191,AFC SUSHI @ SAFEWAY #1205,1451 W COVELL BLVD,DAVIS CA 95616,100.0,d4073f61-cd0c-4348-99e3-b1bf00b385a4
9,FA0010190,AFC SUSHI @ SAFEWAY #1561,2121 COWELL BLVD,DAVIS CA 95616,100.0,e4e2f275-8bac-4bdc-9447-b1d2009cdbcc


Lets write a pipeline. 

In [24]:
def fetch_violations(ProgramId):
    result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections', 
                          params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'ProgramId': ProgramId
    })
    result.raise_for_status()
    results = result.json()
    results_df = pd.DataFrame(results)
    violations = [
        results_df['violations'][0][i]['violation_description'] for i in range(len(results_df['violations'][0]))
    ]
    return(violations)

In [25]:
fetch_violations('PR0000623') # for in-n-out

KeyError: 'violations'

In [ ]:
result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections', 
                          params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'ProgramId': 'PR0000623'
    })

In [ ]:
results = result.json()
results_df = pd.DataFrame(results)

In [ ]:
results_df

In [ ]:
result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections', 
                          params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'ProgramId': 'PR0024103'
    })

In [ ]:
result.text

In [ ]:
def fetch_ProgramId(FacilityID):
    result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/programs', 
                          params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'FacilityID': FacilityID
    })
    result.raise_for_status()
    ProgramId = result.json()[0]['ProgramId']
    return(ProgramId)

In [26]:
fetch_ProgramId('FA0001345')

NameError: name 'fetch_ProgramId' is not defined

In [27]:
def fetch_FacilityID(letter):
    result = requests.post('https://yoloeco.envisionconnect.com/api/pressAgentClient/searchFacilities?', 
                           params  = {
    "PressAgentOid": "c08cb189-894c-4c8c-b595-a5ef010226b4"}, 
                           data = {
    "FacilityName": letter, 
    })
    facility_table = pd.DataFrame(result.json())[['FacilityId', 'FacilityName']]
    return(facility_table)

In [28]:
fetch_FacilityID('A&B LIQUOR')

,FacilityId,FacilityName
0,FA0001345,A&B LIQUOR


In [29]:
import time

In [30]:
[letter for letter in map(chr, range(97, 99))]

['a', 'b']

In [31]:
x = {}
type(x)

dict

In [32]:
def get_violations(): 
    violations = {}
    for letter in map(chr, range(97, 99)): # map(chr, range(97, 123)) takes too long
        time.sleep(0.05) # sleep until making a request for each letter
        facility_table = fetch_FacilityID(letter)
        for index in range(facility_table.shape[0]): # for all facilities returned for this letter
            FacilityId, FacilityName = facility_table.iloc[index]
            time.sleep(0.1) # sleep again for each individual request
            ProgramId = fetch_ProgramId(FacilityId)
            print(FacilityName)
            violations[FacilityName] = fetch_violations(ProgramId)
    return(violations)

In [33]:
violations = get_violations()

NameError: name 'fetch_ProgramId' is not defined

In [ ]:
x = {'key': 'value'}
x['keyy']

In [ ]:
fetch_FacilityID('A&B LIQUOR')

In [ ]:
fetch_ProgramId('FA0001345')

In [ ]:
ProgramId = fetch_ProgramId('FA0001345')            
ProgramId

In [34]:
fetch_violations('PR0000623')

KeyError: 'violations'

In [ ]:
result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections', params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'ProgramId': 'PR0000623'
})
result.raise_for_status()

In [35]:
results = result.json()
results

[{'FacilityId': 'FA0001345',
  'FacilityName': 'A&B LIQUOR',
  'Address': '2328 W CAPITOL AVE ',
  'CityStateZip': 'WEST SACRAMENTO CA 95691 ',
  'LastScore': 100.0,
  'attachmentId': 'c3afdd86-26f1-4cd1-859a-b0f300b80b46'},
 {'FacilityId': 'FA0021935',
  'FacilityName': "AARON'S CHICKEN SHACK",
  'Address': 'YOLO COUNTY FAIR ',
  'CityStateZip': 'WOODLAND CA 95776 ',
  'LastScore': 100.0,
  'attachmentId': 'ac45b3d9-89d1-43ab-909b-b1ce0089ca09'},
 {'FacilityId': 'FA0011391',
  'FacilityName': 'ACAI FRESH',
  'Address': '431 G ST ',
  'CityStateZip': 'DAVIS CA 95616 ',
  'LastScore': 100.0,
  'attachmentId': '170e6176-2b48-4be2-a5fe-b221008fb38f'},
 {'FacilityId': 'FA0022329',
  'FacilityName': 'ACE SUSHI @SAVE MART 604',
  'Address': '1900 ANDERSON RD ',
  'CityStateZip': 'DAVIS CA 95616 ',
  'LastScore': 100.0,
  'attachmentId': '6c94833d-3c62-465c-9b24-b21400b6061d'},
 {'FacilityId': 'FA0019474',
  'FacilityName': 'ACOUSTIC EVENTS',
  'Address': '918 S S ST ',
  'CityStateZip': 'SAC

Lets check this in the browser! 

In [36]:
result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/programs', params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'FacilityID': 'FA0001345'
    }).json()
[result[i]['ProgramId'] for i in range(len(result))]

['PR0000623', 'PR0069422']

In [37]:
def fetch_ProgramId(FacilityID):
    result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/programs', params = {
        'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
        'FacilityID': FacilityID
    }).json()
    ProgramId = [result[i]['ProgramId'] for i in range(len(result))]
    return(ProgramId)

In [38]:
fetch_ProgramId('FA0001345')

['PR0000623', 'PR0069422']

In [39]:
def fetch_violations(ProgramId_list):
    violations = []
    for ProgramId in ProgramId_list: 
        result = requests.get('https://yoloeco.envisionconnect.com/api/pressAgentClient/inspections', params = {
            'PressAgentOid': 'c08cb189-894c-4c8c-b595-a5ef010226b4', 
            'ProgramId': ProgramId
        }).json()
        results_df = pd.DataFrame(result)
        if not results_df.empty: # only append violations if there are any
            violations.extend(
                [results_df['violations'][0][i]['violation_description'] for i in range(len(results_df['violations'][0]))]
            )
    return(violations)

In [40]:
fetch_violations(['PR0000623', 'PR0069422'])

['K006 - Adequate handwashing facilities supplied & accessible',
 'K007 - Proper hot and cold holding temperatures',
 'K021 - Hot and cold water available']

In [41]:
violations = get_violations()

A&B LIQUOR
AARON'S CHICKEN SHACK
ACAI FRESH
ACE SUSHI @SAVE MART 604
ACOUSTIC EVENTS
AFC SUSHI / HOT WOK @ BEL AIR #526
AFC SUSHI / HOT WOK @ RALEY'S #206
AFC SUSHI / HOT WOK @ RALEY'S #448
AFC SUSHI @ SAFEWAY #1205
AFC SUSHI @ SAFEWAY #1561
AFC TIK TOK WOK @ SAFEWAY #1205
AFRIDI FOOD COMPANY
AFTER HOURS BOBA & TEA
AGGIE LIQUOR
AGGIE YOGURT, LLC
AGTECH INNOVATION ALLIANCE
AISLE 1 #2576
AJ HUNDAL MART
AKIRA COFFEE & TEA
ALI BABA RESTAURANT
ALL SEASONS ALL REASONS CATERING
ALOHA POKEE & RAMEN
ALTOS CANTINA
ALYCE NORMAN SCHOOL
AM/PM MINI MARKET #5731- FOOD
ANDERSON FAMILY CATERING BBQ
ANDERSON GAS & MINI MART - FOOD
ANDERSON ROAD SHELL - FOOD
ANDY'S ARCO - FOOD
ANTOJITOS JAIMITO #4TB1720
ANU ANU SNOBALLZ
APNA BAZAAR
APPLEBEE'S - WOODLAND
ARCO AMPM GAS STATION
ARIANA FOOD MARKET
ARMADILLO MUSIC INC.
AROMA CONCESSIONS INC
ASOCIACION LOS CAPORALES - ARENA CONCESSION
AUTHENTIC INDIA
AWM CONCESSIONS PRETZELS
AY! JALISCO TAQUERIA #1
BABA'S CHASKA POINT #4VU9432
BABES.BUBBLES.BOARDS
BABY O'S DON

In [42]:
violations

{'A&B LIQUOR': ['K006 - Adequate handwashing facilities supplied & accessible',
  'K007 - Proper hot and cold holding temperatures',
  'K021 - Hot and cold water available'],
 "AARON'S CHICKEN SHACK": ['K007 - Proper hot and cold holding temperatures'],
 'ACAI FRESH': [],
 'ACE SUSHI @SAVE MART 604': [],
 'ACOUSTIC EVENTS': [],
 'AFC SUSHI / HOT WOK @ BEL AIR #526': [],
 "AFC SUSHI / HOT WOK @ RALEY'S #206": [],
 "AFC SUSHI / HOT WOK @ RALEY'S #448": [],
 'AFC SUSHI @ SAFEWAY #1205': [],
 'AFC SUSHI @ SAFEWAY #1561': [],
 'AFC TIK TOK WOK @ SAFEWAY #1205': ['K033 - Nonfood-contact surfaces clean'],
 'AFRIDI FOOD COMPANY': ['K021 - Hot and cold water available',
  'K026 - Approved thawing methods used, frozen food',
  'K033 - Nonfood-contact surfaces clean'],
 'AFTER HOURS BOBA & TEA': [],
 'AGGIE LIQUOR': ['K045 - Floors, walls and ceilings: built, maintained, and clean'],
 'AGGIE YOGURT, LLC': ['K034 - Warewashing facilities: installed, maintained, used; test strips'],
 'AGTECH INNOVA

#### Safeway

Check the [docs](https://requests.readthedocs.io/en/latest/api/?requests.get)!

In [70]:
url = 'https://www.safeway.com/abs/pub/xapi/pgmsearch/v1/search/products'
params = {
    'request-id': 5561736793073137191,
    'q': 'eggs',
    'rows': 30,
    'start': 0,
    'search-type': 'keyword',
    'storeid': 3132,
    'featured': 'true',
    'url': 'https://www.safeway.com',
    'pageurl': 'https://www.safeway.com', 
    'search-uid': 'uid%3D3640904575678%3Av%3D12.0%3Ats%3D1674581210532%3Ahc%3D3', 
    'pagename': 'search',
    'dvid': 'web-4.1search',
}
header = {
    #'accept': 'application/json, text/plain, */*',
    #'accept-encoding': 'gzip, deflate, br, zstd',
    #'accept-language': 'en-US,en;q=0.9',
    #'cache-control': 'no-cache',
    'Cookie': "visid_incap_1610353=NpQjFt3XShCg5HmsJYLwrxWpGmcAAAAAQUIPAAAAAABTbH3K+b6Xlzm4bbIilz0A; OptanonAlertBoxClosed=2024-10-24T20:13:12.155Z; akacd_PR-bg-www-prod-safeway=3914245866~rv=80~id=f9720057ac5012141d9501502823ae32; nlbi_1610353=M32aK2/TRgycgjWj6eNT2gAAAABGpxl4Fz9uGtERK5SQr0G+; incap_ses_1357_1610353=x9+QQFPW5kOEJ75beQjVEupbhWcAAAAANJR7ORNIqIiNK+nGTcaNmw==; AMCVS_A7BF3BC75245ADF20A490D4D%40AdobeOrg=1; ACI_S_ECommBanner=safeway; abs_gsession=%7B%22info%22%3A%7B%22COMMON%22%3A%7B%22Selection%22%3A%22default%22%2C%22preference%22%3A%22J4U%22%2C%22userType%22%3A%22G%22%2C%22zipcode%22%3A%2294611%22%2C%22banner%22%3A%22safeway%22%2C%22siteType%22%3A%22C%22%2C%22customerType%22%3A%22%22%2C%22resolvedBy%22%3A%22%22%7D%2C%22J4U%22%3A%7B%22zipcode%22%3A%2294611%22%2C%22storeId%22%3A%223132%22%7D%2C%22SHOP%22%3A%7B%22zipcode%22%3A%2294611%22%2C%22storeId%22%3A%223132%22%7D%7D%7D; ACI_S_abs_previouslogin=%7B%22info%22%3A%7B%22COMMON%22%3A%7B%22Selection%22%3A%22default%22%2C%22preference%22%3A%22J4U%22%2C%22userType%22%3A%22G%22%2C%22zipcode%22%3A%2294611%22%2C%22banner%22%3A%22safeway%22%2C%22siteType%22%3A%22C%22%2C%22customerType%22%3A%22%22%2C%22resolvedBy%22%3A%22%22%7D%2C%22J4U%22%3A%7B%22zipcode%22%3A%2294611%22%2C%22storeId%22%3A%223132%22%7D%2C%22SHOP%22%3A%7B%22zipcode%22%3A%2294611%22%2C%22storeId%22%3A%223132%22%7D%7D%7D; SWY_SYND_USER_INFO=%7B%22storeAddress%22%3A%22%22%2C%22storeZip%22%3A%2294611%22%2C%22storeId%22%3A%223132%22%2C%22preference%22%3A%22J4U%22%7D; ACI_S_ECommSignInCount=0; at_check=true; SAFEWAY_MODAL_LINK=; SWY_SHARED_SESSION_INFO=%7B%22info%22%3A%7B%22COMMON%22%3A%7B%22userType%22%3A%22G%22%2C%22zipcode%22%3A%2294611%22%2C%22banner%22%3A%22safeway%22%2C%22preference%22%3A%22J4U%22%2C%22Selection%22%3A%22default%22%2C%22wfcStoreId%22%3A%225799%22%2C%22userData%22%3A%7B%7D%2C%22grsSessionId%22%3A%2265868f2b-f89f-4a7a-9edc-f8607c5728a9%22%2C%22siteType%22%3A%22C%22%2C%22customerType%22%3A%22%22%2C%22resolvedBy%22%3A%22%22%7D%2C%22J4U%22%3A%7B%22storeId%22%3A%223132%22%2C%22zipcode%22%3A%2294611%22%2C%22userData%22%3A%7B%7D%7D%2C%22SHOP%22%3A%7B%22storeId%22%3A%223132%22%2C%22zipcode%22%3A%2294611%22%2C%22userData%22%3A%7B%7D%7D%7D%7D; OptanonConsent=isGpcEnabled=0&datestamp=Mon+Jan+13+2025+10%3A31%3A10+GMT-0800+(Pacific+Standard+Time)&version=202409.1.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=a294f937-8674-4c26-9178-9f1b60f6b59b&interactionCount=2&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0004%3A1%2CC0003%3A1&AwaitingReconsent=false&intType=3&geolocation=US%3BCA; nlbi_1610353_2147483392=5gvNQA0KaE9HVNxe6eNT2gAAAABCup9oWsSAk1ZDDN2TmX8R; reese84=3:ExNoTZgnHBodyq0jNmWs8g==:J4VcV6yDVCf3FV56rwTIU0ZXkTM8sNmwT9oLEQRGS7tecwRkhajCAwJkTzB8jP83kx7XvlxPaSjkfhEOeHL45QvGi6QlLt1eIzP5/FW9ya+sDicCoCaxSCUOJEgQNKLdzYYAhbpZ8c2qMWT4CYUIYE94uC6MQqu7NIoDGszKTcBq6GYnz6j6GgmNCffd6J43IBdcMTMMh+9WMDk3+FAhIQ3p1HpHGPGuy39PSxe1VLglWGMrcmdax81dwmzRsGKkA2BKcq5Zh8bdEZWwHIT02lXZxgglCcRMdZN77huBciKdRmC6Ie9M6CHLYogzRoVgH0QEuaCBDzx+3TzrmimnffHp4fwe2wcraMfyyvzEViz8PPH9hrEJaRiLRb10QfcxsIASJ1oe06T2tl+GFd6LlHO72GKSgm1FyJLpcIQAj7ot8a9k6SJVTd65bLawwhiPUOR9V3Reh1zVU40hUYVYYgK0WQAHT2onMOhwvk4Pic5vdxSO7iKW0HxX+jqZIq79WRw1c/qcVrAFg7veHbQsJw==:qij+BfrUCJWyBDLfyP92iy/QSXKrDa2fVrAr/ygSiSY=; AMCV_A7BF3BC75245ADF20A490D4D%40AdobeOrg=179643557%7CMCIDTS%7C20102%7CMCMID%7C88307423134550140093054020464612948435%7CMCOPTOUT-1736800272s%7CNONE%7CvVersion%7C5.5.0; mbox=PC#ce30384f188648868a639d719bb80350.35_0#1800037874|session#c608f2b05a384af1a61447777b4fc45d#1736794934",
    'Ocp-Apim-Subscription-Key': '5e790236c84e46338f4290aa1050cdd4'
    #'pragma': 'no-cache',
    #'priority': 'u=1, i',
    #'referer': 'https://www.safeway.com/shop/search-results.html?q=eggs',
    #'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
    #'sec-ch-ua-mobile':'?0',
    #'sec-ch-ua-platform':"macOS",
    #'sec-fetch-dest': 'empty',
    #'sec-fetch-mode': 'cors',
    #'sec-fetch-site': 'same-origin',
    #'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
}

# try without cookie and subscription key! 

In [72]:
results = requests.get(url, params = params, headers = header)
results.json()

{'appCode': '[PS: 200]',
 'appMsg': '[PS: Success.]',
 'primaryProducts': {'response': {'numFound': 924,
   'disableTracking': False,
   'start': 0,
   'miscInfo': {'attributionToken': 'wgHwwQoMCKa9lbwGENvg-sYDEAEaJDY3YmZlYjE0LTAwMDAtMjI5ZC1iNjE1LWQ0M2EyY2QxYTAyZioqQUJTWVNfMjBkMGMwMmQtZGJjNC00MzdkLWI3ZWItMmE2MzUxOTBiZGI4Mjia7sYwxcvzF8LwnhXUsp0Vjr6dFba3jC32mYQio4CXIpjWty2b1rctkPeyMI6RyTCTy-kwqOWqLToad2ViLXNhZmV3YXktc2VydmluZy1jb25maWdIAVgBaAF6AnBo',
    'query': 'eggs',
    'filter': '(inventory(3132, price) >0) AND (attributes.is_market_place_product : ANY("false"))'},
   'isExactMatch': True,
   'docs': [{'status': 'active',
     'name': 'Egg Lands Best Bowl Potato Loaded Scramble Breakfast - 7 Oz',
     'pid': '970784376',
     'upc': '0071514121243',
     'id': '970784376',
     'storeId': '3132',
     'featured': True,
     'inventoryAvailable': '1',
     'pastPurchased': False,
     'restrictedValue': '0',
     'salesRank': 99999,
     'agreementId': 0,
     'featuredProductId': 0

In [57]:
s = requests.Session()
s.cookies

<RequestsCookieJar[]>

In [58]:
s.get('https://www.safeway.com/')
s.cookies

<RequestsCookieJar[Cookie(version=0, name='akacd_PR-bg-www-prod-safeway', value='3914246268~rv=40~id=5ff889452a03eb6843d3f752795e161b', port=None, port_specified=False, domain='www.safeway.com', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=None, discard=True, comment=None, comment_url=None, rest={'SameSite': 'None'}, rfc2109=False), Cookie(version=0, name='incap_ses_1357_1610353', value='Af6XHKYiW2QheMBbeQjVEnxdhWcAAAAAKjJeldjJ1YmYJ2ImghDLjw==', port=None, port_specified=False, domain='www.safeway.com', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='nlbi_1610353', value='2vOeNJsJIUY05N7G6eNT2gAAAACRWfgq5GR2qZVrzBgkVskN', port=None, port_specified=False, domain='www.safeway.com', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, exp

In [60]:
r=s.get(url, params=params, headers = {'Ocp-Apim-Subscription-Key': '5e790236c84e46338f4290aa1050cdd4'}) #error! 

KeyboardInterrupt: 

In [69]:
url2 = 'https://www.safeway.com/abs/pub/xapi/search/products'

r = requests.get(url2, params, headers = {'Ocp-Apim-Subscription-Key': 'e914eec9448c4d5eb672debf5011cf8f'}) #error! 
r.json()

{'response': {'numFound': 921,
  'disableTracking': False,
  'start': 0,
  'miscInfo': {'attributionToken': 'wgHwwQoMCMK8lbwGEMy2rLMDEAEaJDY3YzIwYWY1LTAwMDAtMjI1OC05YmFlLTMwZmQzODE2ZWZlOCoqQUJTWVNfZWFhMGNjZmEtZDA1My00OGJhLWE3OGUtMDc3NGI2ZWQzN2RmMjio5aot9pmEIo6-nRXUsp0VwvCeFaOAlyK3t4wtxcvzF5jWty2b1rctkPeyMJruxjCOkckwlMvpMDoad2ViLXNhZmV3YXktc2VydmluZy1jb25maWdIAVgBaAF6AnBo',
   'query': 'eggs',
   'sort': '',
   'filter': '(inventory(3132, price) >0) AND (attributes.is_market_place_product : ANY("false"))'},
  'isExactMatch': True,
  'docs': [{'status': 'active',
    'name': 'Lucerne Farms Eggs Cage Free Large - 18 Count',
    'pid': '960304348',
    'upc': '0002113003154',
    'id': '960304348',
    'storeId': '3132',
    'featured': False,
    'inventoryAvailable': '1',
    'pastPurchased': False,
    'restrictedValue': '0',
    'salesRank': 47,
    'agreementId': 0,
    'featuredProductId': 0,
    'imageUrl': 'https://images.albertsons-media.com/is/image/ABS/960304348',
    'price': 1

### Summary 

- Check the query type, header and params using the developer tools 
- Often, multiple API queries are made to display one result 